###  Code Setup

In [ ]:

from fastapi import FastAPI
from pydantic import BaseModel
import requests
from bs4 import BeautifulSoup

app = FastAPI()


Tool Input Models


In [ ]:
class UniversityQueryInput(BaseModel):
    name: str

class UniversityURLInput(BaseModel):
    url: str

MCP TOOL 1 (LOCAL): Get University Website URL

In [ ]:
@app.post("/get_university_url")
def get_university_url(input: UniversityQueryInput):
    try:
        api_url = f"https://api.data.gov/ed/collegescorecard/v1/schools"
        params = {
            "api_key": "YOUR_API_KEY",  # Replace with your real key
            "school.name": input.name,
            "fields": "school.name,school.school_url",
            "per_page": 1
        }
        response = requests.get(api_url, params=params)
        data = response.json()
        if data.get("results"):
            return {"url": "https://" + data["results"][0]["school.school_url"]}
        return {"url": ""}
    except Exception as e:
        return {"error": str(e)}

get_university_url_schema = {
    "name": "get_university_url",
    "description": "Finds the admissions URL for a given U.S. university name.",
    "parameters": {
        "type": "object",
        "properties": {
            "name": {
                "type": "string",
                "description": "The official name of the university."
            }
        },
        "required": ["name"]
    }
}

MCP TOOL 2 (LOCAL): Extract Deadlines & Application Info

In [ ]:
def extract_deadline_info(input: UniversityURLInput):
    try:
        response = requests.get(input.url, timeout=10)
        soup = BeautifulSoup(response.text, 'html.parser')
        text = soup.get_text()
        lines = text.split('\n')
        keywords = ["deadline", "apply", "application", "due"]
        extracted = [line.strip() for line in lines if any(k in line.lower() for k in keywords)]
        return {"info": extracted[:5]}  # limit to top 5 relevant lines
    except Exception as e:
        return {"error": str(e)}

extract_deadline_info_schema = {
    "name": "extract_deadline_info",
    "description": "Extracts application deadline and instructions from a university website.",
    "parameters": {
        "type": "object",
        "properties": {
            "url": {
                "type": "string",
                "description": "The admissions URL of the university."
            }
        },
        "required": ["url"]
    }
}

MCP TOOL 3 (HOSTED): University Ranking Info (example hosted endpoint)

In [ ]:
@app.post("/get_ranking")
def get_ranking(input: UniversityQueryInput):
    simulated_ranks = {
        "UT Austin": "Ranked #10 for Computer Science in the U.S.",
        "Stanford": "Ranked #1 for Computer Science in the U.S.",
        "Berkeley": "Ranked #2 for Computer Science in the U.S."
    }
    return {"ranking": simulated_ranks.get(input.name, "Ranking not found.")}

get_ranking_schema = {
    "name": "get_ranking",
    "description": "Returns the university's CS ranking based on the name provided.",
    "parameters": {
        "type": "object",
        "properties": {
            "name": {
                "type": "string",
                "description": "The official name of the university."
            }
        },
        "required": ["name"]
    }
}

# MCP Agent Setup (LangChain)

In [ ]:
from langchain.agents.mcp import McpAgentExecutor
from langchain.chat_models import ChatOpenAI

agent = McpAgentExecutor.from_mcp_server(
    mcp_server_url="http://localhost:3333",
    llm=ChatOpenAI()
)

# Example prompt
response = agent.invoke(
    "Tell me about Stanford University’s application deadlines, how to apply, and its ranking."
)
print(response)